# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = pd.read_csv('../WeatherPy/output_files/citydata.csv')
weather_df = pd.DataFrame(weather_file)
clean_weather_df = weather_df.drop(columns="Unnamed: 0")
clean_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Paine,-33.82,-70.75,46.40,87,75,2.24,CL,1592529966
1,Port Hedland,-20.32,118.57,73.40,64,0,8.05,AU,1592529913
2,Goodlettsville,36.32,-86.71,79.00,53,75,11.41,US,1592529967
3,Bira,49.00,132.47,65.14,65,87,1.16,RU,1592529967
4,Bluff,-46.60,168.33,46.92,69,0,11.32,NZ,1592529779
...,...,...,...,...,...,...,...,...,...
581,Roald,62.58,6.13,64.00,77,23,12.75,NO,1592530082
582,Staraya Toropa,56.28,31.67,62.58,82,0,2.82,RU,1592530082
583,Constantine,36.37,6.61,62.60,77,40,4.70,DZ,1592530082
584,Sarankhola,22.31,89.79,84.31,81,100,4.99,BD,1592530082


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure maps with unique API key
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_file[['Lat', 'Lng']]

weight = weather_file['Humidity']


In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '750px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [5]:
# Assign the marker layer to a variable

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# Add the layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather = clean_weather_df.loc[(clean_weather_df['Max Temp'] <=75) & (clean_weather_df['Max Temp'] >=70) &
                               (clean_weather_df["Wind Speed"] <=10) & (clean_weather_df["Cloudiness"] ==0)]


ideal_weather.dropna()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Port Hedland,-20.32,118.57,73.40,64,0,8.05,AU,1592529913
154,Barra do Corda,-5.50,-45.25,72.05,78,0,1.21,BR,1592529995
164,Awjilah,29.11,21.29,74.68,28,0,4.72,LY,1592529997
176,Qaryat Sulūq,31.67,20.25,70.79,32,0,0.60,LY,1592529999
191,Mossamedes,-15.20,12.15,71.15,59,0,3.83,AO,1592529923
315,Aginskoye,51.10,114.53,70.11,45,0,2.44,RU,1592530028
338,Benghazi,32.12,20.07,74.44,48,0,3.06,LY,1592529877
392,Lac-Mégantic,45.58,-70.88,73.99,59,0,2.95,CA,1592529949
423,Zhigansk,66.77,123.37,72.05,39,0,6.85,RU,1592529925
427,Morros,-9.45,-46.30,70.61,69,0,5.99,BR,1592530052


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df
hotel_df = ideal_weather.sort_index(inplace=False)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Port Hedland,-20.32,118.57,73.40,64,0,8.05,AU,1592529913
154,Barra do Corda,-5.50,-45.25,72.05,78,0,1.21,BR,1592529995
164,Awjilah,29.11,21.29,74.68,28,0,4.72,LY,1592529997
176,Qaryat Sulūq,31.67,20.25,70.79,32,0,0.60,LY,1592529999
191,Mossamedes,-15.20,12.15,71.15,59,0,3.83,AO,1592529923
315,Aginskoye,51.10,114.53,70.11,45,0,2.44,RU,1592530028
338,Benghazi,32.12,20.07,74.44,48,0,3.06,LY,1592529877
392,Lac-Mégantic,45.58,-70.88,73.99,59,0,2.95,CA,1592529949
423,Zhigansk,66.77,123.37,72.05,39,0,6.85,RU,1592529925
427,Morros,-9.45,-46.30,70.61,69,0,5.99,BR,1592530052


In [8]:
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = " "
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Port Hedland,-20.32,118.57,73.40,64,0,8.05,AU,1592529913,
154,Barra do Corda,-5.50,-45.25,72.05,78,0,1.21,BR,1592529995,
164,Awjilah,29.11,21.29,74.68,28,0,4.72,LY,1592529997,
176,Qaryat Sulūq,31.67,20.25,70.79,32,0,0.60,LY,1592529999,
191,Mossamedes,-15.20,12.15,71.15,59,0,3.83,AO,1592529923,
315,Aginskoye,51.10,114.53,70.11,45,0,2.44,RU,1592530028,
338,Benghazi,32.12,20.07,74.44,48,0,3.06,LY,1592529877,
392,Lac-Mégantic,45.58,-70.88,73.99,59,0,2.95,CA,1592529949,
423,Zhigansk,66.77,123.37,72.05,39,0,6.85,RU,1592529925,
427,Morros,-9.45,-46.30,70.61,69,0,5.99,BR,1592530052,


In [9]:
# Set parameters to search for hotels within 5000 meters 
#geocoordinates
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# Sset up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



In [10]:
# Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    # Get city name, lat, lnt 
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    country = row["Country"]
    
    # Add coordinates to params
    params["location"] = f"{lat},{lng}"

    # Create url and make API call
    try:
        print(f"For {index} {city_name} in {country}.")
        response = requests.get(base_url, params=params).json()
    
    # extract results
        results = response['results']
    
    # save the hotel name to dataframe
    
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show blank
    except:
        print("Hotel not found... skipping.")
        
    print("------------")    

For 1 Port Hedland in AU.
Closest hotel is The Esplanade Hotel.
------------
For 154 Barra do Corda in BR.
Closest hotel is Hotel Diniz.
------------
For 164 Awjilah in LY.
Hotel not found... skipping.
------------
For 176 Qaryat Sulūq in LY.
Hotel not found... skipping.
------------
For 191 Mossamedes in AO.
Closest hotel is Hotel ibis Styles Iu Namibe.
------------
For 315 Aginskoye in RU.
Closest hotel is Gostinitsa Gobi.
------------
For 338 Benghazi in LY.
Closest hotel is Tibesti Hotel.
------------
For 392 Lac-Mégantic in CA.
Closest hotel is Manoir D'Orsennens.
------------
For 423 Zhigansk in RU.
Closest hotel is Gostinichnyy Kompleks Aylgy.
------------
For 427 Morros in BR.
Hotel not found... skipping.
------------
For 441 Agía Galíni in GR.
Closest hotel is Sky Beach Hotel.
------------
For 475 Montes Altos in BR.
Closest hotel is Hotel e Restaurante Manaaim.
------------
For 483 Zhangjiakou in CN.
Closest hotel is Blue Whale Hotel.
------------
For 527 Dingli in MT.
Closes

In [11]:
# Store the first Hotel result into the DataFrame
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Port Hedland,-20.32,118.57,73.40,64,0,8.05,AU,1592529913,The Esplanade Hotel
154,Barra do Corda,-5.50,-45.25,72.05,78,0,1.21,BR,1592529995,Hotel Diniz
164,Awjilah,29.11,21.29,74.68,28,0,4.72,LY,1592529997,
176,Qaryat Sulūq,31.67,20.25,70.79,32,0,0.60,LY,1592529999,
191,Mossamedes,-15.20,12.15,71.15,59,0,3.83,AO,1592529923,Hotel ibis Styles Iu Namibe
315,Aginskoye,51.10,114.53,70.11,45,0,2.44,RU,1592530028,Gostinitsa Gobi
338,Benghazi,32.12,20.07,74.44,48,0,3.06,LY,1592529877,Tibesti Hotel
392,Lac-Mégantic,45.58,-70.88,73.99,59,0,2.95,CA,1592529949,Manoir D'Orsennens
423,Zhigansk,66.77,123.37,72.05,39,0,6.85,RU,1592529925,Gostinichnyy Kompleks Aylgy
427,Morros,-9.45,-46.30,70.61,69,0,5.99,BR,1592530052,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer on top of heat map
marker = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…